## Importing Torch, and Personalised Scripts, using Conda env ml_adi

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as td
import numpy as np
import os
import random
from PIL import Image
from scripts.unet_model import *
from scripts.indices import *

## Loading the Data

In [2]:
image_dir = '/application/pi/Desktop/all imp data/Unet_training/images/'
mask_dir = '/application/pi/Desktop/all imp data/Unet_training/masks/'

In [3]:
class PlaneDataSet(td.Dataset):
    
    def __init__(self, image_path, mask_path):
        import os
        
        self.image_path = image_path
        
        self.mask_path = mask_path
        c = os.listdir(mask_path)
        c.remove('.ipynb_checkpoints')
        self.filenames = c
        
        self.filenames 
    def __getitem__(self, index):
        
        #Load the image
        img_path = os.path.join(self.image_path, self.filenames[index])
        image =  np.load(img_path)
        
        # Make Indices
        image_np = allIndices(image)
        
        # Normalize
        image_np = normalize(image_np)
        
        # Convert to a tensor
        image_tensor = torch.Tensor(image_np)
        
        # Now doing operations to mask
        mask_path = os.path.join(self.mask_path, self.filenames[index])
        mask =  np.load(mask_path)
        mask_np = np.expand_dims(mask, axis=0)
        mask_tensor = torch.Tensor(mask_np)
        
        return(image_tensor, mask_tensor)
    
    def __len__(self):
        return len(self.filenames)

In [4]:
def load_data(image_path, mask_path):

    # Load all of the images, transforming them
    dataset = PlaneDataSet(image_path , mask_path)
    index_chosen = [78,210,87,276,89,175,298,20,129,198,283,233,54,191,300,26,170,76,154,309,379,376,266,82,287,174,345,311,44,217,121,409,449,374,277,199,97,293,143,11,347,436,156,226,378,153,28,100,218,341,326,336,202,354,124,407,109,292,184,476,183,360,232,235,269,369,141,75,270,102,10,138,0,380,415,201,59,268,145,459,384,302,131,337,456,123,322,25,328,117,231,248,401,356,45,128]
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    test_dataset1 = torch.utils.data.Subset(dataset,indices=index_chosen)

    # define the loaders for the image data
    image_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=4,
        num_workers=0,
        shuffle=True,
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=4,
        num_workers=0,
        shuffle=True,
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=4,
        num_workers=0,
        shuffle=True,
    )
    test_loader1 = torch.utils.data.DataLoader(
        test_dataset1,
        batch_size=4,
        num_workers=0,
        shuffle=False,
    )
    return image_loader, train_loader, test_loader,test_loader1

image_loader, train_loader, test_loader,test_loader1 = load_data(image_dir, mask_dir)

In [20]:
torch.save(train_loader, 'train_loader.pth')
torch.save(test_loader, 'test_loader.pth')

In [5]:
train_loader =torch.load('train_loader.pth')
test_loader = torch.load('test_loader.pth')

# Defining Unet and Training for 1 epoch

In [8]:
def train(model, device, data_loader, optimizer, epoch):
    
    model.to(device)
    model.train()
    
    train_loss = 0
    print("Epoch:", epoch, "...")
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(data_loader):
        
        
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
        output = model(data)
        
        # Get the loss
        loss = loss_criteria(output, target)
        
        # Keep a running total
        train_loss += loss.item()
        # Backpropagate
        loss.backward()
        optimizer.step()
        print('loss =', loss.item(),' number',batch_idx)
        
    # return average loss for the epoch
    avg_training_loss = train_loss / (batch_idx+1)
    print("\tTraining set: Average loss: {:.6f}".format(avg_training_loss))
    return avg_training_loss


In [ ]:
# To check if all epochs ran 
thisSimulationRan =np.array(['this', 'frikkin','works']) 
# Create a UNet model

model = UNet(n_channels=20, n_classes = 1)

# Use the best available device (GPU/CPU) for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # Uncomment the below when training continued epochs
model.load_state_dict(torch.load('/application/pi/unet_trained_with_indices.pt', map_location=device))

#Specify the optimizer and  loss criteria
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
loss_criteria = nn.BCELoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []

epochs = 5
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        
torch.save(model.state_dict(), '/application/pi/unet_trained_with_indices.pt')
np.save('thisSimulationRan2.npy',thisSimulationRan)

model.eval()




Training on cpu
Epoch: 1 ...
loss = 0.14375798404216766  number 0
loss = 1.144284725189209  number 1
loss = 0.3585912585258484  number 2
loss = 0.3325299620628357  number 3
loss = 0.3292069137096405  number 4
loss = 0.3197952210903168  number 5
loss = 0.3057980537414551  number 6
loss = 0.24334707856178284  number 7
loss = 0.3274212181568146  number 8
loss = 0.21803642809391022  number 9
loss = 0.3145124912261963  number 10
loss = 0.21049872040748596  number 11
loss = 0.2615402936935425  number 12
loss = 0.2242359220981598  number 13
loss = 0.25480812788009644  number 14
loss = 0.21243591606616974  number 15
loss = 0.15965889394283295  number 16
loss = 0.24524901807308197  number 17
loss = 0.22708536684513092  number 18
loss = 0.1885732114315033  number 19
loss = 0.20344752073287964  number 20
loss = 0.20828869938850403  number 21
loss = 0.13487860560417175  number 22
loss = 0.1763792634010315  number 23
loss = 0.24195390939712524  number 24
loss = 0.1974869966506958  number 25
loss = 

loss = 0.11280934512615204  number 24
loss = 0.1462761014699936  number 25
loss = 0.24212640523910522  number 26
loss = 0.3200082778930664  number 27
loss = 0.2649799585342407  number 28
loss = 0.14702780544757843  number 29
loss = 0.15361842513084412  number 30
loss = 0.22734525799751282  number 31
loss = 0.15559306740760803  number 32
loss = 0.1407974809408188  number 33
loss = 0.17059403657913208  number 34
loss = 0.1829776167869568  number 35
loss = 0.17095947265625  number 36
loss = 0.1920025497674942  number 37
loss = 0.18715600669384003  number 38
loss = 0.11405115574598312  number 39
loss = 0.17605526745319366  number 40
loss = 0.20076516270637512  number 41
loss = 0.1753508448600769  number 42
loss = 0.23658421635627747  number 43
loss = 0.16854265332221985  number 44
loss = 0.19771617650985718  number 45
loss = 0.1567152589559555  number 46
loss = 0.10047101229429245  number 47
loss = 0.1306172013282776  number 48
loss = 0.14487531781196594  number 49
loss = 0.120379053056240

In [12]:
i = 0
model = UNet(n_channels=20, n_classes = 1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.load_state_dict(torch.load('/application/pi/unet_trained_with_indices.pt', map_location=device))

for batch_idx, (data, target) in enumerate(test_loader):
    data1 = data
    target1 = target
    if batch_idx%1 ==0:
        with torch.no_grad():
            output = model(data1)
        a=np.array(output[0])
        b=np.array(target1[0])
        
        np.save('/application/pi/Desktop/all imp data/Unet testing/images/trial_0_'+str(i)+'.npy',a)
        np.save('/application/pi/Desktop/all imp data/Unet testing/masks/trial_0_'+str(i)+'.npy',b)
        
        a=np.array(output[1])
        b=np.array(target1[1])
        
        np.save('/application/pi/Desktop/all imp data/Unet testing/images/trial_1_'+str(i)+'.npy',a)
        np.save('/application/pi/Desktop/all imp data/Unet testing/masks/trial_1_'+str(i)+'.npy',b)
        
        a=np.array(output[2])
        b=np.array(target1[2])
        
        np.save('/application/pi/Desktop/all imp data/Unet testing/images/trial_2_'+str(i)+'.npy',a)
        np.save('/application/pi/Desktop/all imp data/Unet testing/masks/trial_2_'+str(i)+'.npy',b)
        
        a=np.array(output[3])
        b=np.array(target1[3])
        
        np.save('/application/pi/Desktop/all imp data/Unet testing/images/trial_3_'+str(i)+'.npy',a)
        np.save('/application/pi/Desktop/all imp data/Unet testing/masks/trial_3_'+str(i)+'.npy',b)
        
        
        
        
        i = i+1
    print(batch_idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [1]:
2+2

4